In [171]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from collections import OrderedDict
from operator import itemgetter
import statistics


%matplotlib inline

In [9]:
df = pd.read_csv('boardgame-elite-users.csv')


#kmeans = KMeans(n_clusters=2, random_state=0).fit(df)
#kmeans.labels_



In [10]:
df.columns = ['userID', 'gameID', 'rating']

In [11]:
titles = pd.read_csv('boardgame-titles.csv')

In [12]:
titles.columns = ['gameID', 'title']
df = pd.merge(df, titles, how='inner', on=['gameID'])
games = df['gameID'].unique().tolist()
users_l = df['userID'].unique().tolist()


In [208]:
test.get_recommendations()

[0.98412845167041652,
 0.73412845167041652,
 0.09375808130004612,
 0.015631866742986222,
 -0.23438365905622716,
 0.81573787381012397,
 0.52412845167041655,
 0.30561634094377288,
 -0.60624191869995392,
 1.1341284516704164,
 -0.16587154832958353,
 0.19780743677539486,
 0.59375808130004615,
 0.21259511833708328,
 0.33412845167041649,
 0.22737797079084662,
 0.34375808130004615,
 -0.16587154832958353,
 -0.61013568656603256,
 1.0156163409437728,
 0.096892492351375203,
 0.47412845167041651,
 -0.18536476835493404,
 0.2283221089402776,
 -0.66453483391482016,
 -0.32114324093776025,
 0.81815722864883367,
 -0.065871548329583529,
 -0.22253934426196312,
 0.68412845167041647,
 -0.61587154832958357,
 0.26133872646133643,
 0.88412845167041643,
 1.8125951183370832,
 -0.23860235323269938,
 -0.03310750764862478,
 -0.34890523365317827,
 0.36875808130004611,
 0.52490407089019686,
 -0.052631130211116608,
 -0.084611918063314834,
 0.47964511792219006,
 -0.1099685382961388,
 0.063165872755190297,
 0.05273722970

In [207]:
class user:
    
    def __init__(self, _id):
        
        self.user_id = _id
        
        self.rated_games = df[df['userID'] == _id].iloc[:,1:].sort_values('rating', ascending=False)
        self.ratings = dict(zip(self.rated_games['gameID'].tolist(), self.rated_games['rating'].tolist()))

        self.avg = np.mean(self.rated_games['rating'])

        self.ratings_dict =  dict(zip(games,[self.ratings[i] - self.avg if i in self.ratings.keys() else 0 for i in games]))
        self.ratings_vector = [np.array(list(self.ratings_dict.values()))]
    
    def get_neighbors(self, K=10, show_similarities=False):
        d = dict()
        [d.update({i : cosine_similarity(self.ratings_vector, users[i].ratings_vector)[0][0]}) for i in users]
        od = OrderedDict(sorted(d.items(), key=itemgetter(1)))
        od.popitem()
        
        neighbors = []
        
        for i in range(0, K):
            if(show_similarities):
                neighbors.append(od.popitem())
            else:
                neighbors.append(users[od.popitem()[0]])
        return neighbors
    
    
    def get_recommendations(self):
        neighbors = self.get_neighbors()
        averages = []
        
        for i in range(0, len(self.ratings_vector[0])):
            ratings = []
            [ratings.append(x.ratings_vector[0][i]) for x in neighbors]
            averages.append(statistics.mean(ratings))
        return averages
        
            
    
    
        
test = user(191116)

users = dict()

[users.update({i : user(i)}) for i in users_l ]






        

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [183]:
def get_recommendations(a, rsm=False, comp=False):
    d = dict()
    [d.update({i : cosine_similarity(users[a].ratings_vector, users[i].ratings_vector)[0][0]}) for i in users]
    print(d)
    od = OrderedDict(sorted(d.items(), key=itemgetter(1)))
    
    od.popitem()
    other = od.popitem()
    other_ratings = users[other[0]].rated_games
    
    s1 = pd.merge(other_ratings, users[a].rated_games, how='inner', on=['gameID'])
    s1['error'] = abs(s1['rating_x'] - s1['rating_y'])
    
    s2 = pd.merge(other_ratings, users[a].rated_games, how='outer', on=['gameID'], indicator=True)
    s2 = s2[s2._merge == 'left_only']
    
    recs = s2.head(10).title_x

    print('user_id: ' , users[a].user_id)
    
    if(rsm):
        return s1['error'].mean()
    elif(comp):
        print(s1.drop(['title_x', 'gameID', 'error'], axis=1))
    else:
        print(recs)
    
    #sns.regplot(x='rating_x', y='rating_y', data = s1)
    #sns.heatmap(s1['rating_x', 'rating_y'])
    
    #print(s1)
    
#get_recommendations(test.user_id, rsm=True)

#rsms = list()

#[rsms.append(get_recommendations(i, rsm=True)) for i in users_l]

#print(rsms)

#users[272].ratings_vector